In [1]:
from inputs import *
from datetime import datetime, timedelta
import random
import pandas as pd
import math

In [2]:
def generate_timetable(line, total_trips, start_time="05:50", min_interval=5, max_interval=15,
                       interval_bias=8, travel_time_range=(70, 75)):
    """
    Gera um timetable sintético para um CP (ponto de controle), incluindo os intervalos entre viagens.

    Args:
        cp_id (str): identificador do CP.
        num_trips (int): número de partidas a gerar.
        start_time (str): hora da primeira viagem.
        min_interval (int): intervalo mínimo entre viagens.
        max_interval (int): intervalo máximo.
        interval_bias (int): valor central mais provável.
        travel_time_range (tuple): tempo de viagem (min, max)

    Returns:
        pd.DataFrame com columns = ['trip_id', 'cp', 'departure_time', 'departure_interval', 'planned_travel_time']
    """
    current_time = datetime.strptime(start_time, "%H:%M")
    trips = []
    int_div = total_trips // 2
    mod = total_trips % 2
    num_trips_tuple = (int_div, int_div + mod)
    cps = ("cp1", "cp2")
    start_times_tuple = (current_time, current_time + timedelta(minutes=10))

    for num_trips, cp, time, idx in zip(num_trips_tuple, cps, start_times_tuple, (1,0)):
        for i in range(1, num_trips + 1):
            if i == 1:
                interval = None
            else:
                interval = int(random.gauss(interval_bias, 2))
                interval = max(min_interval, min(interval, max_interval))
                time += timedelta(minutes=interval)

            travel_time = float(random.randint(*travel_time_range))

            trips.append({
                'trip_id': f"l{line}_{cp}_{i}",
                'line': line,
                'start_cp': cp,
                'start_cp_id': f"{cp}_l{line}",
                'dest_cp': cps[idx],
                'dest_cp_id': f"{cps[idx]}_l{line}",
                'departure_time': time,
                'departure_interval': interval,
                'planned_travel_time': travel_time
            })

    return pd.DataFrame(trips)

def haversine_distance(latlon1, latlon2):

    R = 6371

    lat1, lon1 = latlon1[1], latlon1[0]
    lat2, lon2 = latlon2[1], latlon2[0]

    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c * 1.2

    return distance

def summarize_cp_locations(lines_info):
    cp_locations_summary = {}
    for line_id, line_data in lines_info.items():
        for cp in list(line_data.keys())[:2]:
            cp_id = f"{cp}_l{line_id}"
            latlon = line_data[cp]["latlon"]
            cp_locations_summary[cp_id] = latlon
    return cp_locations_summary

def calculate_cp_distances(cp_locations_summary, lines_info):
    from_distances = {}

    # Criar um mapeamento auxiliar: cp_id -> line
    cp_to_line = {}
    for line_id, data in lines_info.items():
        for cp_key in ["cp1", "cp2"]:
            cp_id = f"{cp_key}_l{line_id}"
            cp_to_line[cp_id] = line_id

    for from_cp, from_coords in cp_locations_summary.items():
        to_distances = {}
        for to_cp, to_coords in cp_locations_summary.items():
            if from_cp == to_cp:
                to_distances[to_cp] = 0.0
                continue

            from_line = cp_to_line.get(from_cp)
            to_line = cp_to_line.get(to_cp)

            # Se são da mesma linha e são pares opostos, usa a distância fornecida
            if from_line == to_line:
                cp_keys = [k.split("_")[0] for k in (from_cp, to_cp)]
                if set(cp_keys) == {"cp1", "cp2"}:
                    to_distances[to_cp] = lines_info[from_line]["cp_distance_km"]
                    continue

            # Caso contrário, calcula com Haversine
            to_distances[to_cp] = haversine_distance(from_coords, to_coords)

        from_distances[from_cp] = to_distances

    return from_distances


def transform_cp_depot_distances(original_dict):
    inverse_dict = {}

    for cp, depot_distances in original_dict.items():
        for depot, distance in depot_distances.items():
            if depot not in inverse_dict:
                inverse_dict[depot] = {}
            inverse_dict[depot][cp] = distance

    combined_dict = {**original_dict, **inverse_dict}

    return combined_dict

def merge_distances_dicts(dict_1, dict_2):
    merged_dict = {}

    for key, distances in dict_1.items():
        merged_dict[key] = distances.copy()
        if key in dict_2:
            for subkey, fallback_distance in dict_2[key].items():
                if subkey not in merged_dict[key]:
                    merged_dict[key][subkey] = fallback_distance

    return merged_dict

def make_deadhead_df(deadhead_dict):
    sorted_dict = {k:v for k, v in sorted(deadhead_dict.items(), key=lambda item: item[0])}
    for k in list(sorted_dict.keys()):
        sorted_subdict = {_k: _v for _k, _v in sorted(sorted_dict[k].items(), key=lambda item: item[0])}
        sorted_dict[k] = sorted_subdict
    deadhead_df = pd.DataFrame(sorted_dict)
    return deadhead_df

def make_deadhead_times_df(avg_speed_kmh, deadhead_df):
    conversor = lambda x: x/avg_speed_kmh*60
    deadhead_times_df = deadhead_df.apply(conversor)
    return deadhead_times_df


In [3]:
cp_locations_summary = summarize_cp_locations(lines_info)
cp_distances = calculate_cp_distances(cp_locations_summary, lines_info)
transformed_cp_depot_distances = transform_cp_depot_distances(cp_depot_distances)
dh_dict = merge_distances_dicts(transformed_cp_depot_distances, cp_distances)
dh_df = make_deadhead_df(dh_dict)
dh_times_df = make_deadhead_times_df(20, dh_df)

In [4]:
dh_df

,cp1_l4,cp1_l59,cp1_l60,cp2_l4,cp2_l59,cp2_l60,d1_4,d1_59,d2_4,d2_59,d2_60
cp1_l4,0.000000,9.422740,9.422740,24.000000,3.549382,9.501166,0.00,4.08,28.80,14.76,3.24
cp1_l59,9.422740,0.000000,0.000000,14.778975,11.400000,17.722110,4.08,0.00,15.84,13.68,6.60
cp1_l60,9.422740,0.000000,0.000000,14.778975,11.086324,19.000000,4.08,0.00,15.84,13.68,6.60
cp2_l4,24.000000,14.778975,14.778975,0.000000,4.077578,3.237288,28.80,16.32,0.00,5.64,23.04
cp2_l59,3.549382,11.400000,11.086324,4.077578,0.000000,6.675498,14.76,13.68,5.64,0.00,22.80
cp2_l60,9.501166,17.722110,19.000000,3.237288,6.675498,0.000000,3.24,6.60,23.04,22.80,0.00
d1_4,0.000000,4.080000,4.080000,28.800000,14.760000,3.240000,NaN,NaN,NaN,NaN,NaN
d1_59,4.080000,0.000000,0.000000,16.320000,13.680000,6.600000,NaN,NaN,NaN,NaN,NaN
d2_4,28.800000,15.840000,15.840000,0.000000,5.640000,23.040000,NaN,NaN,NaN,NaN,NaN
d2_59,14.760000,13.680000,13.680000,5.640000,0.000000,22.800000,NaN,NaN,NaN,NaN,NaN


In [5]:
dh_times_df

,cp1_l4,cp1_l59,cp1_l60,cp2_l4,cp2_l59,cp2_l60,d1_4,d1_59,d2_4,d2_59,d2_60
cp1_l4,0.000000,28.268221,28.268221,72.000000,10.648146,28.503498,0.00,12.24,86.40,44.28,9.72
cp1_l59,28.268221,0.000000,0.000000,44.336924,34.200000,53.166331,12.24,0.00,47.52,41.04,19.80
cp1_l60,28.268221,0.000000,0.000000,44.336924,33.258972,57.000000,12.24,0.00,47.52,41.04,19.80
cp2_l4,72.000000,44.336924,44.336924,0.000000,12.232734,9.711863,86.40,48.96,0.00,16.92,69.12
cp2_l59,10.648146,34.200000,33.258972,12.232734,0.000000,20.026495,44.28,41.04,16.92,0.00,68.40
cp2_l60,28.503498,53.166331,57.000000,9.711863,20.026495,0.000000,9.72,19.80,69.12,68.40,0.00
d1_4,0.000000,12.240000,12.240000,86.400000,44.280000,9.720000,NaN,NaN,NaN,NaN,NaN
d1_59,12.240000,0.000000,0.000000,48.960000,41.040000,19.800000,NaN,NaN,NaN,NaN,NaN
d2_4,86.400000,47.520000,47.520000,0.000000,16.920000,69.120000,NaN,NaN,NaN,NaN,NaN
d2_59,44.280000,41.040000,41.040000,16.920000,0.000000,68.400000,NaN,NaN,NaN,NaN,NaN


In [11]:
float(dh_times_df.loc["cp1_l4", "cp1_l59"])

28.268220860607236

In [7]:
tt_l4 = generate_timetable("4", 290)

tt_l59 = generate_timetable("59", 100)

tt_l60 = generate_timetable("60", 120)

timetables = pd.concat([tt_l4, tt_l59, tt_l60], ignore_index=True)

timetables['covered'] = False

In [ ]:
timetables.tail(12)

,trip_id,line,start_cp,start_cp_id,dest_cp,dest_cp_id,departure_time,departure_interval,planned_travel_time,covered
0,l4_cp1_1,4,cp1,cp1_l4,cp2,cp2_l4,1900-01-01 05:50:00,NaN,70.0,False
1,l4_cp1_2,4,cp1,cp1_l4,cp2,cp2_l4,1900-01-01 05:56:00,6.0,74.0,False
2,l4_cp1_3,4,cp1,cp1_l4,cp2,cp2_l4,1900-01-01 06:09:00,13.0,75.0,False
3,l4_cp1_4,4,cp1,cp1_l4,cp2,cp2_l4,1900-01-01 06:14:00,5.0,75.0,False
4,l4_cp1_5,4,cp1,cp1_l4,cp2,cp2_l4,1900-01-01 06:23:00,9.0,70.0,False
5,l4_cp1_6,4,cp1,cp1_l4,cp2,cp2_l4,1900-01-01 06:33:00,10.0,74.0,False
6,l4_cp1_7,4,cp1,cp1_l4,cp2,cp2_l4,1900-01-01 06:39:00,6.0,70.0,False
7,l4_cp1_8,4,cp1,cp1_l4,cp2,cp2_l4,1900-01-01 06:46:00,7.0,72.0,False
8,l4_cp1_9,4,cp1,cp1_l4,cp2,cp2_l4,1900-01-01 06:58:00,12.0,73.0,False
9,l4_cp1_10,4,cp1,cp1_l4,cp2,cp2_l4,1900-01-01 07:08:00,10.0,75.0,False


In [148]:
timetables.to_csv("timetables.csv")

In [ ]:
def get_earliest_trip(timetables: pd.DataFrame) -> dict:
    
    uncovered = timetables[timetables['covered'] == False]
    
    if uncovered.empty:
        result = False
    else:
        min_departure_time = uncovered['departure_time'].min()
        earliest_uncovered = uncovered[uncovered['departure_time'] == min_departure_time]
        if earliest_uncovered.empty:
            result = False
        else:
            result = earliest_uncovered.iloc[0]
    
    return result

def get_nearest_depot(cp_id: str, depots: dict, cp_depot_distances: dict) -> str:
    for k in cp_depot_distances[cp_id].keys():
        if depots[k]['departed'] == depots[k]['capacity']:
            continue
        else:
            return k
    print("No depots found!")
    return False

def select_next_trip(timetables: pd.DataFrame, last_trip_id: str) -> str:
    
    def find_tmax(line):
        df = timetables[(timetables.line == line) & ~(timetables.covered)]['departure_interval']
        return df.max()
    
    ti = timetables[timetables.trip_id == last_trip_id].iloc[0]
    ti_line = ti.line
    eti = ti.departure_time + timedelta(minutes=ti.planned_travel_time)    
    tmax = timetables[(timetables.line == ti.line) & ~(timetables.covered)]["departure_interval"].max()
    tj_start_cp = ti.dest_cp_id
    tj_candidates = timetables[
        (
            (timetables.start_cp_id == tj_start_cp)
                & ~(timetables.covered)
                & (timetables.departure_time >= eti)
                & (timetables.departure_time <= eti + timedelta(minutes=tmax))
        )
    ]
    tj = tj_candidates.sample()

    tj = pd.DataFrame({'a':[], 'b':[]})

    if tj.empty:

        other_lines = list(timetables[timetables.covered == False]['line'].unique())
        other_lines.remove(ti_line)
        tmax_per_line = {l: find_tmax(l) for l in other_lines}
        
        tj_pre_candidates = timetables[
            (
                (timetables.line.isin(other_lines))
                    & ~(timetables.covered)
            )
        ].copy()

        tj_pre_candidates['dh_time'] = tj_pre_candidates.apply(
            lambda row: timedelta(minutes=dh_times_df.loc[tj_start_cp, row['start_cp_id']]), axis=1
        )

        tj_pre_candidates['lower_bound'] = eti + tj_pre_candidates['dh_time']

        tj_pre_candidates['tmax'] = tj_pre_candidates.apply(
            lambda row: timedelta(minutes=tmax_per_line[row['line']]), axis=1
        )

        tj_pre_candidates['upper_bound'] = (
            eti
                + tj_pre_candidates['dh_time']
                + tj_pre_candidates['tmax']
        )

        tj_candidates = tj_pre_candidates[
            (
                (tj_pre_candidates.departure_time <= tj_pre_candidates['upper_bound'])
                    & (tj_pre_candidates.departure_time >= tj_pre_candidates['lower_bound'])
            )
        ]

        tj = tj_candidates[tj_candidates.dh_time == tj_candidates.dh_time.min()]

    return tj.iloc[0]




In [154]:
print(select_random_t_j(timetables, "l4_cp1_1"))

trip_id                                l60_cp2_11
line                                           60
start_cp                                      cp2
start_cp_id                               cp2_l60
dest_cp                                       cp1
dest_cp_id                                cp1_l60
departure_time                1900-01-01 07:14:00
departure_interval                            6.0
planned_travel_time                          73.0
covered                                     False
dh_time                    0 days 00:09:42.711790
lower_bound            1900-01-01 07:09:42.711790
tmax                              0 days 00:13:00
upper_bound            1900-01-01 07:22:42.711790
Name: 460, dtype: object


In [ ]:
dist = 0
time = 0
columns = []
max_time = 360
max_dist = 120

def initialize(timetables, depots, cp_depot_distances, lines_info):
    route = []
    if not (t_i := get_earliest_trip(timetables)):
        return
    departure_depot = get_nearest_depot(t_i["cp_id"], depots, cp_depot_distances)
    depots[departure_depot]['departed'] += 1
    route.append(departure_depot)
    route.append(t_i['trip_id'])
    dist += lines_info[t_i["line"]]["cp_distance_km"]
    time += t_i['departure_interval'] + t_i['planned_travel_time']
    
    return route




{'trip_id': 'l4_cp1_147', 'line': '4', 'cp': 'cp1', 'cp_id': 'cp1_l4', 'departure_time': '00:02:00', 'departure_interval': np.float64(6.0), 'planned_travel_time': np.int64(73), 'covered': np.False_}
28.8
79.0


In [17]:
timetables

,trip_id,cp,departure_time,departure_interval,planned_travel_time,covered
0,CP1_L4_1,CP1_L4,05:50:00,NaN,75,False
1,CP1_L4_2,CP1_L4,05:56:00,6.0,72,False
2,CP1_L4_3,CP1_L4,06:01:00,5.0,71,False
3,CP1_L4_4,CP1_L4,06:11:00,10.0,74,False
4,CP1_L4_5,CP1_L4,06:16:00,5.0,74,False
...,...,...,...,...,...,...
1015,CP2_L60_116,CP2_L60,20:37:00,5.0,74,False
1016,CP2_L60_117,CP2_L60,20:46:00,9.0,70,False
1017,CP2_L60_118,CP2_L60,20:53:00,7.0,72,False
1018,CP2_L60_119,CP2_L60,21:01:00,8.0,74,False


In [5]:
import networkx as nx
from collections import deque

# 1. Example graph with three attributes per edge
sample_graph = {
    'A': {
        'B': {'reduced_cost': 1.2, 'dist': 10, 'time': 5},
        'C': {'reduced_cost': 2.5, 'dist': 15, 'time': 7},
    },
    'B': {
        'C': {'reduced_cost': 1.0, 'dist': 8, 'time': 4},
        'D': {'reduced_cost': 3.1, 'dist': 12, 'time': 6},
    },
    'C': {
        'D': {'reduced_cost': 0.9, 'dist': 5, 'time': 3},
    },
    'D': {}   # no outgoing edges
}

# 2. Conversion function
def dict_to_nx_multi(graph_dict, directed=True):
    G = nx.DiGraph() if directed else nx.Graph()
    for u, nbrs in graph_dict.items():
        for v, attrs in nbrs.items():
            # **unpack** all three attributes onto the edge
            G.add_edge(u, v,
                       reduced_cost=attrs['reduced_cost'],
                       dist=attrs['dist'],
                       time=attrs['time'])
    return G

# build the NetworkX graph
G = dict_to_nx_multi(sample_graph)

# 3. Inspect
for u, v, data in G.edges(data=True):
    print(f"{u} → {v}  :  reduced_cost={data['reduced_cost']}, dist={data['dist']}, time={data['time']}")


A → B  :  reduced_cost=1.2, dist=10, time=5
A → C  :  reduced_cost=2.5, dist=15, time=7
B → C  :  reduced_cost=1.0, dist=8, time=4
B → D  :  reduced_cost=3.1, dist=12, time=6
C → D  :  reduced_cost=0.9, dist=5, time=3


In [20]:
for nbr in G.neighbors("A"):
    print(G["A"][nbr]['reduced_cost'])

1.2
2.5


In [ ]:
def run_spfa(graph, source_node):
    red_costs = {source_node: 0}
    red_costs.update({k: math.inf for k in graph.keys() if k != source_node})
    dist = {source_node: 0}
    dist.update({k: math.inf for k in graph.keys() if k != source_node})
    time = {source_node: 0}
    time.update({k: math.inf for k in graph.keys() if k != source_node})
    queue = deque()
    queue.append(source_node)
    while len(queue) > 0:
        u = queue.popleft()
        current_time = graph[u]["start_time"]
        start_time = graph[u]["start_time"]
        planned_travel_time = graph[u]["planned_travel_time"]
        end_time = graph[u]["end_time"]
        start_cp = graph[u]['start_cp_id']
        end_cp = graph[u]['dest_cp_id']

        for nbr in graph.neighbors(u):
            arc_reduced_cost = graph[u][nbr]["reduced_cost"]
            arc_time = graph[u][nbr]["time"]
            arc_dist = graph[u][nbr]["dist"]
            if red_costs[u] + arc_reduced_cost < red_costs[v]:
                red_costs[v] = red_costs[u] + dv
                if v not in queue:
                    queue.append(v)
        print(red_costs)
    return red_costs

trip_nodes = [n for n, attr in graph.nodes(data=True) if attr.get("type") == "T"]

NameError: name 'graph' is not defined